In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.datasets import BinaryLabelDataset
from sklearn.ensemble import RandomForestClassifier



def evaluate_model(data, cutoff_score=60, dependent_variable='math score'):
    """
    Step 4 - training classifier and get fairness metrics.
                # dependent variable : math score ( 60 as cutoff for binary classification)
                # protected class: Gender
                # privileged/unprivileged groups associated: Male/Female
                # two fairness metrics calculated : SPD and DI
                # classifier used: RandomForestClassifier

    Parameters:
    - data: The dataset including the protected attribute 'gender'. 
    - cutoff_score: The cutoff for the binary classification. Defaults to 60.
    - Depenent_variable: dependent variable choose for this step. Default to math score.
    
    Returns:
    - accuracy: Accuracy of the classifier.
    - spd: Statistical Parity Difference.
    - di: Disparate Impact.
    """

    # Prepare the features and target variables
    X = data.drop([dependent_variable], axis=1)
    y = (data[dependent_variable] >= cutoff_score).astype(int)

    # One-hot encoding for categorical columns
    categorical_columns = ['gender', 'race/ethnicity', 'parental level of education', 'lunch', 'test preparation course']
    X = pd.get_dummies(X, columns=categorical_columns)
    
    # Split the dataset into training and testing datasets 80/20
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Train the classifier
    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)
    
    # Predict on the test set
    y_pred = clf.predict(X_test)
    
    # Compute the accuracy
    accuracy = accuracy_score(y_test, y_pred)
    
    # Create a BinaryLabelDataset for aif360 metrics
    test_dataset = BinaryLabelDataset(df=pd.concat((X_test, y_test), axis=1),
                                      label_names=[dependent_variable], #outcome of interest
                                      protected_attribute_names=['gender_male'], #protected attribute
                                      unprivileged_protected_attributes=[0]) #value within the protected attribute is considered unprivileged

    test_pred_dataset = test_dataset.copy()
    test_pred_dataset.labels = y_pred.reshape(-1, 1) # predict y from test data used to calculated fairness metrics

    # Compute fairness metrics
    privileged_groups = [{'gender_male': 1}]
    unprivileged_groups = [{'gender_male': 0}]
    
    metric = BinaryLabelDatasetMetric(test_pred_dataset,
                                      unprivileged_groups=unprivileged_groups,
                                      privileged_groups=privileged_groups)
    spd = metric.statistical_parity_difference()
    di = metric.disparate_impact()
    print(f"Accuracy: {accuracy}")
    print(f"Statistical Parity Difference: {spd}")
    print(f"Disparate Impact: {di}")
    return accuracy, spd, di

In [7]:
original_data = pd.read_csv(r'./data/student_performance_prediction_dataset.csv').drop(['reading score','writing score'], axis=1)
# reweight_data = pd.read_csv() 

print('original_data  (no reweighing) result as below:')
accuracy, spd, di = evaluate_model(data=original_data)
 
# print('Reweigh data result as below:')
# accuracy, spd, di = evaluate_model(data=reweight_data)

original_data  (no reweighing) result as below:
Accuracy: 0.71
Statistical Parity Difference: -0.11729323308270678
Disparate Impact: 0.8675721561969439
